In [ ]:
!pip install -U transformers datasets accelerate evaluate


In [ ]:
from google.colab import files
import pandas as pd
import io
uploaded = files.upload()
file_name = next(iter(uploaded))
df = pd.read_excel(io.BytesIO(uploaded[file_name]))
df.to_csv("armenian_propaganda_dataset.csv", index=False)
print(df.head())

In [3]:
import re
from sklearn.model_selection import train_test_split
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    return text
df['cleaned_text'] = df['text'].apply(clean_text)
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'], df['label'], test_size=0.2, random_state=42
)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
model = LogisticRegression()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)
print(f"Ճշգրտություն: {accuracy_score(y_test, y_pred) * 100:.2f}%")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred, target_names=["Չեզոք (0)", "Քարոզչական (1)"]))

In [ ]:
def predict_propaganda(text):
    text_cleaned = clean_text(text)
    text_vec = vectorizer.transform([text_cleaned])
    prediction = model.predict(text_vec)[0]
    return "Քարոզչական (1)" if prediction == 1 else "Չեզոք (0)"

print(predict_propaganda("ՀԱՊԿ-ից դուրս գալը անհրաժեշտ է"))
print(predict_propaganda("Երևանում անցկացվեց AI հակոն"))

In [6]:
from imblearn.over_sampling import RandomOverSampler
vectorizer = TfidfVectorizer(max_features=1000)
X_vec = vectorizer.fit_transform(df['cleaned_text'])
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_vec, df['label'])
X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, class_weight='balanced')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=500)
X_new = selector.fit_transform(X_resampled, y_resampled)
X_train, X_test, y_train, y_test = train_test_split(
    X_new, y_resampled, test_size=0.2, random_state=42
)
model.fit(X_train, y_train)

In [ ]:
from xgboost import XGBClassifier

model = XGBClassifier(
    scale_pos_weight=sum(y_resampled==0)/sum(y_resampled==1),
    eval_metric='logloss'
)
model.fit(X_train, y_train)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
scale_pos_weight = sum(y_train == 0) / sum(y_train == 1)
model = XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    scale_pos_weight=scale_pos_weight,
    eval_metric='logloss',
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred, target_names=["Չեզոք (0)", "Քարոզչական (1)"]))

In [ ]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import numpy as np
y_probs = model.predict_proba(X_test)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_test, y_probs)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
y_pred_optimized = (y_probs >= optimal_threshold).astype(int)
print(classification_report(y_test, y_pred_optimized))

from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300]
}

grid = GridSearchCV(XGBClassifier(), param_grid, cv=3, scoring='f1')
grid.fit(X_train, y_train)
print("Լավագույն պարամետրեր:", grid.best_params_)

In [ ]:
!pip install catboost
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report

rf = RandomForestClassifier(
    class_weight='balanced',
    n_estimators=200,
    max_depth=5,
    random_state=42
)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Results:")
print(classification_report(y_test, y_pred_rf))

cb = CatBoostClassifier(
    auto_class_weights='Balanced',
    iterations=200,
    depth=5,
    verbose=0,
    random_state=42
)
cb.fit(X_train, y_train)
y_pred_cb = cb.predict(X_test)
print("\nCatBoost Results:")
print(classification_report(y_test, y_pred_cb))


In [ ]:
cb = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    auto_class_weights='Balanced',
    verbose=0
)
from transformers import BertTokenizer, BertForSequenceClassification
import joblib
joblib.dump(cb, 'best_propaganda_model.pkl')

In [ ]:
from catboost import CatBoostClassifier
import joblib
cb = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    auto_class_weights='Balanced',
    verbose=0
)
cb.fit(X_train, y_train)
joblib.dump({'model': cb, 'vectorizer': vectorizer}, 'propaganda_model.pkl')

In [ ]:
import joblib

saved_data = joblib.load('propaganda_model.pkl')
loaded_model = saved_data['model']
vectorizer = saved_data['vectorizer']

def predict_text(text):
    cleaned_text = clean_text(text)
    text_vec = vectorizer.transform([cleaned_text])
    prediction = loaded_model.predict(text_vec)[0]
    return "Քարոզչական" if prediction == 1 else "Չեզոք"
print(predict_text("ՀԱՊԿ-ից դուրս գալը պետք է դառնա առաջնահերթություն"))
print(predict_text("Երևանում անցկացվեց տեխնոլոգիական փառատոն"))

In [ ]:
cb.save_model('catboost_model.cbm')
from catboost import CatBoost
loaded_model = CatBoost()
loaded_model.load_model('catboost_model.cbm')

In [ ]:
print(f"Մոդելը ուսուցված է: {loaded_model.is_fitted()}")
print(f"Օգտագործված հատկությունների քանակ: {loaded_model.feature_names_}")

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import joblib

def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

X_train_texts, X_test_texts, y_train, y_test = train_test_split(
    df['cleaned_text'],
    df['label'],
    test_size=0.2,
    random_state=42
)

vectorizer = TfidfVectorizer(max_features=500)
X_train_vec = vectorizer.fit_transform(X_train_texts)
X_test_vec = vectorizer.transform(X_test_texts)
cb = CatBoostClassifier(
    iterations=500,
    depth=6,
    learning_rate=0.05,
    auto_class_weights='Balanced',
    verbose=0
)
cb.fit(X_train_vec, y_train)
joblib.dump({'model': cb, 'vectorizer': vectorizer}, 'propaganda_model_full.pkl')
def predict_text(text):
    cleaned_text = clean_text(text)
    text_vec = vectorizer.transform([cleaned_text])
    prediction = cb.predict(text_vec)[0]
    return "Քարոզչական" if prediction == 1 else "Չեզոք"

print(predict_text("ՀԱՊԿ-ից դուրս գալը պետք է դառնա առաջնահերթություն"))
print(predict_text("Երևանում անցկացվեց տեխնոլոգիական փառատոն"))
feature_importances = cb.get_feature_importance()
top_features = sorted(zip(vectorizer.get_feature_names_out(), feature_importances),
                 key=lambda x: x[1], reverse=True)[:10]
print("Top Features:", top_features)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

class_weights = [1, len(y_train[y_train==0])/len(y_train[y_train==1])]

cb = CatBoostClassifier(
    iterations=1000,
    depth=5,
    learning_rate=0.05,
    loss_function='Logloss',
    eval_metric='F1',
    early_stopping_rounds=20,
    class_weights=class_weights,
    verbose=100
)

cb.fit(X_train_vec, y_train, eval_set=(X_test_vec, y_test))

y_pred = cb.predict(X_test_vec)

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Չեզոք (0)', 'Քարոզչական (1)'],
            yticklabels=['Չեզոք (0)', 'Քարոզչական (1)'])
plt.xlabel('Կանխատեսված')
plt.ylabel('Փաստացի')
plt.title('Confusion Matrix')
plt.show()


wrong_indices = np.where(y_test != y_pred)[0][:5]

for i, idx in enumerate(wrong_indices):
    print(f"\nՕրինակ {i+1}:")
    print(f"Տեքստ: {X_test_texts.iloc[idx]}")
    print(f"Կանխատեսում: {'Քարոզչական' if y_pred[idx] == 1 else 'Չեզոք'}")
    print(f"Փաստացի: {'Քարոզչական' if y_test.iloc[idx] == 1 else 'Չեզոք'}")

In [ ]:
!pip install nlpaug
!pip install transformers

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

In [ ]:
!pip install nlpaug transformers
!python -m nltk.downloader wordnet omw-1.4

import random
from nlpaug.augmenter.word import SynonymAug
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier

def armenian_augment(text, n=2):
    """Custom augmentation for Armenian text"""
    try:
        aug = SynonymAug(
            aug_src='wordnet',
            lang='hye',
            aug_p=0.3,
            stopwords=['և', 'է', 'ը', 'որ', 'ինչ']
        )
        return [aug.augment(text) for _ in range(n)]
    except:
        replacements = {
            "օրինակ": ["նմուշ", "պատկեր"],
            "տեքստ": ["գրառում", "հոդված"],
            "ՀԱՊԿ": ["Հայաստանի անվտանգության պայմանագիր", "ռազմական դաշինք"],
            "ադրբեջան": ["ադրբեջանական կողմ", "հարևան երկիր"]
        }
        results = []
        for _ in range(n):
            augmented = text
            for word, subs in replacements.items():
                if word in augmented:
                    augmented = augmented.replace(word, random.choice(subs))
            results.append(augmented)
        return results

print(armenian_augment("օրինակ տեքստ ՀԱՊԿ-ի մասին", n=2))

vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1,2),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]+\b'
)

cb = CatBoostClassifier(
    iterations=500,
    depth=4,
    learning_rate=0.03,
    l2_leaf_reg=5,
    early_stopping_rounds=10,
    eval_metric='F1',
    class_weights=[1, 3],
    verbose=100
)

try:
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained("Geotrend/bert-base-hy")
    print("BERT tokenizer loaded successfully for Armenian")
except:
    print("Proceeding with CatBoost only")

In [ ]:
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from nlpaug.augmenter.word import SynonymAug
def armenian_augment(text, n=2):
    """Improved augmentation with Armenian-specific rules"""
    armenian_synonyms = {
        "ՀԱՊԿ": ["Հայաստանի անվտանգության պայմանագիր", "ռազմական դաշինք", "ՀԱՊԿ կազմակերպություն"],
        "ադրբեջան": ["ադրբեջանական կողմ", "հարևան պետություն", "Ադրբեջանի Հանրապետություն"],
        "փաշինյան": ["վարչապետ", "նիկոլ փաշինյան", "կառավարության ղեկավար"],
        "զինված": ["ռազմական", "պաշտպանական", "զինվորական"],
        "հարձակում": ["ագրեսիա", "հրաձգություն", "զինված բախում"]
    }

    results = []
    for _ in range(n):
        augmented = text
        for word, subs in armenian_synonyms.items():
            if word in augmented:
                augmented = augmented.replace(word, random.choice(subs))
        results.append(augmented)
    return results

print(armenian_augment("ՀԱՊԿ-ից դուրս գալու մասին հայտարարություն", n=2))

vectorizer = TfidfVectorizer(
    max_features=1500,
    ngram_range=(1, 3),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]{3,}\b',
    stop_words=['և', 'է', 'ը', 'որ', 'ինչ', 'պետք', 'էր']
)
cb = CatBoostClassifier(
    iterations=800,
    depth=5,
    learning_rate=0.025,
    l2_leaf_reg=3,
    early_stopping_rounds=15,
    eval_metric='F1',
    class_weights=[1, 4],
    text_features=['text'],
    verbose=200
)
try:
    from transformers import pipeline
    bert_classifier = pipeline(
        "text-classification",
        model="Geotrend/bert-base-hy",
        tokenizer="Geotrend/bert-base-hy"
    )
    print("BERT model loaded successfully")
except Exception as e:
    print(f"Couldn't load BERT model: {e}")

In [ ]:
import random
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from nlpaug.augmenter.word import SynonymAug

def armenian_augment(text, n=2):
    """Improved augmentation with varied outputs"""
    armenian_synonyms = {
        "ՀԱՊԿ": ["Հայաստանի անվտանգության պայմանագիր", "ռազմական դաշինք", "ՀԱՊԿ կազմակերպություն"],
        "ադրբեջան": ["ադրբեջանական կողմ", "հարևան պետություն", "Ադրբեջանի Հանրապետություն"],
        "փաշինյան": ["վարչապետ", "նիկոլ փաշինյան", "կառավարության ղեկավար"],
        "զինված": ["ռազմական", "պաշտպանական", "զինվորական"],
        "հարձակում": ["ագրեսիա", "հրաձգություն", "զինված բախում"],
        "դուրս գալ": ["հեռանալ", "լքել", "դադարել մասնակցել"]
    }

    results = []
    words = text.split()
    for _ in range(n):
        augmented = []
        for word in words:
            if word in armenian_synonyms and random.random() < 0.5:
                augmented.append(random.choice(armenian_synonyms[word]))
            else:
                augmented.append(word)
        results.append(' '.join(augmented))
    return results
print(armenian_augment("ՀԱՊԿ-ից դուրս գալու մասին հայտարարություն", n=3))
vectorizer = TfidfVectorizer(
    max_features=1500,
    ngram_range=(1, 3),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]{3,}\b',
    stop_words=['և', 'է', 'ը', 'որ', 'ինչ', 'պետք', 'էր']
)

cb = CatBoostClassifier(
    iterations=800,
    depth=5,
    learning_rate=0.025,
    l2_leaf_reg=3,
    early_stopping_rounds=15,
    eval_metric='F1',
    class_weights=[1, 4],
    verbose=200
)
try:
    from transformers import pipeline
    bert_classifier = pipeline(
        "text-classification",
        model="bert-base-multilingual-cased",
        tokenizer="bert-base-multilingual-cased"
    )
    print("Multilingual BERT model loaded successfully")
except Exception as e:
    print(f"Couldn't load BERT model: {e}")

In [ ]:
import random
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

data = {
    'text': [
        'ՀԱՊԿ-ից դուրս գալը վտանգավոր է',
        'Երևանում տեխնոլոգիական փառատոն',
        'Փաշինյանի հայտարարությունը վատն է',
        'Դպրոցներում նոր ծրագիր է մտցվել'
    ],
    'label': [1, 0, 1, 0]
}
df = pd.DataFrame(data)

def armenian_augment(text, n=3):
    synonyms = {
        "ՀԱՊԿ": ["Հայաստանի անվտանգության պայմանագիր", "ռազմական դաշինք"],
        "փաշինյան": ["վարչապետ", "նիկոլ փաշինյան"],
        "դուրս գալ": ["հեռանալ", "լքել"]
    }
    results = []
    for _ in range(n):
        words = text.split()
        augmented = [random.choice(synonyms.get(word, [word])) for word in words]
        results.append(' '.join(augmented))
    return results

print("Աւգմենտացիայի օրինակ:")
augmented = armenian_augment("ՀԱՊԿ-ից դուրս գալը վտանգավոր է", n=2)
for i, text in enumerate(augmented, 1):
    print(f"{i}. {text}")

vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    stop_words=['և', 'է', 'ը', 'որ']
)

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.25)
X_train_vec = vectorizer.fit_transform(X_train)

cb = CatBoostClassifier(
    iterations=300,
    depth=4,
    learning_rate=0.05,
    early_stopping_rounds=10,
    verbose=50
)

print("\nՄոդելի ուսուցում սկսված է...")
cb.fit(X_train_vec, y_train)
print("Մոդելի ուսուցում ավարտված է")

def save_models_fast():
    joblib.dump({
        'model': cb,
        'vectorizer': vectorizer
    }, 'fast_model.pkl', compress=3)
    print("\nՄոդելը հաջողությամբ պահպանված է fast_model.pkl ֆայլում")

save_models_fast()

test_text = "ՀԱՊԿ-ից հեռանալը վատ գաղափար է"
test_vec = vectorizer.transform([test_text])
prediction = cb.predict(test_vec)[0]
print(f"\nՓորձարկում՝ '{test_text}'")
print("Կանխատեսում:", "Քարոզչական" if prediction == 1 else "Չեզոք")

In [ ]:
import random
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

data = {
    'text': [
        'ՀԱՊԿ-ից դուրս գալը վտանգավոր է',
        'Երևանում տեխնոլոգիական փառատոն',
        'Փաշինյանի հայտարարությունը վտանգավոր է',
        'Դպրոցներում նոր ծրագիր է մտցվել',
        'Ադրբեջանը խախտում է հրադադարը',
        'Հայաստանում տնտեսական աճ է գրանցվում',
        'Ռազմական դաշինքը պետք է լուծարվի',
        'Երևանում մշակութային միջոցառում'
    ],
    'label': [1, 0, 1, 0, 1, 0, 1, 0]
}
df = pd.DataFrame(data)

def armenian_augment(text, n=3):
    synonyms = {
        "ՀԱՊԿ": ["Հայաստանի անվտանգության պայմանագիր", "ռազմական դաշինք", "ՀԱՊԿ կազմակերպություն"],
        "ադրբեջան": ["ադրբեջանական կողմ", "հարևան պետություն", "Ադրբեջանի Հանրապետություն"],
        "փաշինյան": ["վարչապետը", "նիկոլ փաշինյան", "կառավարության ղեկավարը"],
        "վտանգավոր": ["վնասակար", "սպառնալից", "վտանգ ներկայացնող"],
        "դուրս գալ": ["հեռանալ", "լքել", "դադարել մասնակցել"],
        "լուծարվի": ["դադարի գոյություն ունենալ", "փակվի", "վերացվի"]
    }

    results = []
    for _ in range(n):
        words = text.split()
        augmented = []
        for word in words:
            clean_word = word.strip('.,!?()[]{}"\'')
            if clean_word in synonyms:
                replacement = random.choice(synonyms[clean_word])
                if word[-1] in '.,!?()[]{}"\'':
                    replacement += word[-1]
                augmented.append(replacement)
            else:
                augmented.append(word)
        results.append(' '.join(augmented))
    return list(set(results))

print("=== Աւգմենտացիայի Օրինակ ===")
sample_text = "ՀԱՊԿ-ից դուրս գալը վտանգավոր է:"
augmented_samples = armenian_augment(sample_text, n=3)
for i, sample in enumerate(augmented_samples, 1):
    print(f"{i}. {sample}")
vectorizer = TfidfVectorizer(
    max_features=1500,
    ngram_range=(1, 2),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]{3,}\b',
    stop_words=['և', 'է', 'ը', 'որ', 'ինչ', 'պետք', 'էր', 'են', 'մենք']
)

X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.25, random_state=42
)

cb = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=0.03,
    l2_leaf_reg=3,
    early_stopping_rounds=20,
    eval_metric='F1',
    class_weights=[1, 3],
    task_type='CPU',
    verbose=100
)

print("\n=== Մոդելի Ուսուցում ===")
X_train_vec = vectorizer.fit_transform(X_train)
cb.fit(X_train_vec, y_train)
X_test_vec = vectorizer.transform(X_test)
y_pred = cb.predict(X_test_vec)
print("\n=== Գնահատման Արդյունքներ ===")
print(classification_report(y_test, y_pred, target_names=["Չեզոք", "Քարոզչական"]))
print(f"Ճշգրտություն: {accuracy_score(y_test, y_pred):.2f}")

def predict_text(text, show_proba=True):
    """Կանխատեսման ֆունկցիա հավանականություններով"""
    vec = vectorizer.transform([text])
    prediction = cb.predict(vec)[0]

    if show_proba:
        proba = cb.predict_proba(vec)[0]
        print(f"\nՏեքստ: '{text}'")
        print(f"Հավանականություններ: Չեզոք={proba[0]:.2f}, Քարոզչական={proba[1]:.2f}")

    return "Քարոզչական" if prediction == 1 else "Չեզոք"

print("\n=== Փորձարկում ===")
test_cases = [
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր է",
    "Երևանում կայանալու է երաժշտական փառատոն",
    "Ադրբեջանը խախտում է հրադադարի ռեժիմը"
]

for text in test_cases:
    print(f"{text} → {predict_text(text)}")
def save_model():
    joblib.dump({
        'model': cb,
        'vectorizer': vectorizer,
        'predict_func': predict_text,
        'augment_func': armenian_augment
    }, 'armenian_propaganda_detector_v2.pkl')
    print("\n=== Մոդելը պահպանված է ===")

save_model()

In [ ]:
import csv
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from catboost import CatBoostClassifier
import joblib

data = [

    ("ՀԱՊԿ-ից դուրս գալը վտանգավոր հետևանքներ կունենա։", 1),
    ("Ադրբեջանը պարբերաբար խախտում է հրադադարի պայմանավորվածությունը։", 1),
    ("Փաշինյանի կառավարությունը կործանում է երկիրը։", 1),
    ("Ադրբեջանական կողմը սադրանքներ է կազմակերպում։", 1),
    ("ՀԱՊԿ-ից հեռանալը կհանգեցնի անվտանգության կորստին։", 1),
    ("Փաշինյանը չի կարողանում պաշտպանել Հայաստանի շահերը։", 1),
    ("Ռազմական դաշինքը պարտադիր է մեր անվտանգության համար։", 1),
    ("Ադրբեջանը սպառնում է նոր պատերազմով։", 1),
    ("ՀԱՊԿ-ն մեր միակ անվտանգ երաշխիքն է։", 1),
    ("Փաշինյանի հրաժարականը անհրաժեշտ է երկրի փրկության համար։", 1),

    ("Երևանում այս շաբաթ արևոտ եղանակ է սպասվում։", 0),
    ("Հայաստանում տեղի կունենա երաժշտական փառատոն։", 0),
    ("Գյումրիում բացվել է նոր ցուցահանդես։", 0),
    ("Մարզիկները պատրաստվում են միջազգային մրցաշարին։", 0),
    ("Նոր ֆիլմի պրեմիերան մեծ հետաքրքրություն է առաջացրել։", 0),
    ("Դպրոցներում սկսվել են ամառային արձակուրդները։", 0),
    ("Երևանում բացվել է ժամանակակից արվեստի կենտրոն։", 0),
    ("Կենտրոնական պուրակում տնկվել են նոր ծառեր։", 0),
    ("Տեղի ունեցավ գարնանային տոնական միջոցառում։", 0),
    ("Հանրապետությունում արձանագրվել է զբոսաշրջության աճ։", 0),

]

with open("armenian_dataset.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["text", "label"])
    writer.writerows(data)

print("✅ armenian_dataset.csv ֆայլը ստեղծվեց!")

df = pd.read_csv("armenian_dataset.csv")
print("\nDataset preview:")
print(df.head())

vectorizer = TfidfVectorizer(
    max_features=1500,
    ngram_range=(1, 2),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]{3,}\b',
    stop_words=['և', 'է', 'ը', 'որ', 'ինչ', 'պետք', 'էր', 'են', 'մենք']
)

X = vectorizer.fit_transform(df['text'])
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

cb = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=0.03,
    l2_leaf_reg=3,
    eval_metric='Accuracy',
    verbose=100
)

print("\n🚀 Մոդելի ուսուցումը սկսվում է...")
cb.fit(X_train, y_train)
print("✅ Մոդելի ուսուցումը ավարտվեց!")

y_pred = cb.predict(X_test)
print("\n=== Գնահատման արդյունքներ ===")
print(classification_report(y_test, y_pred, target_names=["Չեզոք", "Քարոզչական"]))
print(f"Ճշգրտություն: {accuracy_score(y_test, y_pred):.2f}")

joblib.dump({'model': cb, 'vectorizer': vectorizer}, "propaganda_model.pkl")
print("\n✅ Մոդելը պահպանվեց propaganda_model.pkl ֆայլում!")

def predict_text(text):
    vec = vectorizer.transform([text])
    pred = cb.predict(vec)[0]
    return "Քարոզչական" if pred == 1 else "Չեզոք"

print("\n=== Փորձարկում ===")
test_cases = [
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր է",
    "Երևանում բացվել է նոր ցուցահանդես",
    "Ադրբեջանը սպառնում է նոր պատերազմով",
    "Ադրբեջանը հաղորդում է որ շուտով հարձակվելու է Հայաստանի վրա",
    "Ալիևն ասել է որ սիրում է Ադրբեջանը"
]
for t in test_cases:
    print(f"{t} → {predict_text(t)}")


In [ ]:
import random
import joblib
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

propaganda_texts = [
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր հետևանքներ կունենա։",
    "Ադրբեջանը պարբերաբար խախտում է հրադադարի պայմանավորվածությունը։",
    "Փաշինյանի կառավարությունը կործանում է երկիրը։",
    "Ադրբեջանական կողմը սադրանքներ է կազմակերպում։",
    "ՀԱՊԿ-ից հեռանալը կհանգեցնի անվտանգության կորստին։",
    "Փաշինյանը չի կարողանում պաշտպանել Հայաստանի շահերը։",
    "Ռազմական դաշինքը պարտադիր է մեր անվտանգության համար։",
    "Ադրբեջանը սպառնում է նոր պատերազմով։",
    "ՀԱՊԿ-ն մեր միակ անվտանգ երաշխիքն է։",
    "Փաշինյանի հրաժարականը անհրաժեշտ է երկրի փրկության համար։",
    "Ադրբեջանական զորքերը խախտում են սահմանները։",
    "Ազգային անվտանգությունը վտանգված է Փաշինյանի սխալների պատճառով։",
    "ՀԱՊԿ-ից դուրս գալը կնվազեցնի մեր պաշտպանունակությունը։",
    "Փաշինյանը վտանգում է պետության ապագան։",
    "Ադրբեջանի սադրանքները անպատիժ չեն մնա։",
    "Ռազմական ուժերի հզորացումը կարևոր է մեր գոյատևման համար։",
    "ՀԱՊԿ-ի օգնությունը կենսական նշանակություն ունի։",
    "Փաշինյանի քայլերը վնասում են բանակին։",
    "Ադրբեջանը նախապատրաստվում է նոր ագրեսիայի։",
    "Փաշինյանը կործանարար քաղաքականություն է վարում։",
    "ՀԱՊԿ-ն միակ ուժն է, որ պաշտպանում է մեզ։",
    "Ադրբեջանը խախտում է միջազգային օրենքները։",
    "Փաշինյանի պատճառով երկիրը վտանգված է։",
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր է մեր ապագայի համար։",
    "Ադրբեջանական սպառնալիքները աճում են։",
    "Ռազմական դաշինքը միակ երաշխիքն է խաղաղության։",
    "Փաշինյանի կառավարությունը պետք է հրաժարական տա։",
    "Ադրբեջանը շարունակում է սադրանքները։",
    "ՀԱՊԿ-ից հեռանալը հավասարազոր է պարտության։",
    "Փաշինյանը կործանում է ազգային արժեքները։",
    "Ադրբեջանի ագրեսիան անընդունելի է։",
    "ՀԱՊԿ-ը միակ պաշտպանն է։",
    "Փաշինյանի սխալները աղետալի են։",
    "Ադրբեջանական բանակը սպառնում է մեր սահմաններին։",
    "ՀԱՊԿ-ից դուրս գալը անխոհեմ քայլ է։",
    "Փաշինյանը վնասում է Հայաստանի անվտանգությանը։",
    "Ադրբեջանը վտանգ է ներկայացնում։",
    "ՀԱՊԿ-ից դուրս գալը կհանգեցնի պատերազմի։",
    "Փաշինյանը չի կարողանում ղեկավարել երկիրը։",
    "Ադրբեջանը չի հարգում խաղաղությունը։",
    "ՀԱՊԿ-ի աջակցությունը անհրաժեշտ է։",
    "Փաշինյանը պետք է հրաժարական տա։",
    "Ադրբեջանը խախտում է Հայաստանի սահմանները։",
    "ՀԱՊԿ-ից հեռանալը վտանգավոր է։",
    "Փաշինյանը սխալ որոշումներ է կայացնում։",
    "Ադրբեջանի սադրանքները շարունակվում են։",
    "ՀԱՊԿ-ն կարևոր է մեր պաշտպանության համար։",
    "Փաշինյանը վտանգավոր քաղաքականություն է վարում։",
    "Ադրբեջանը սպառնում է մեր անվտանգությանը։",
    "ՀԱՊԿ-ից դուրս գալը սխալ է։",
    "Փաշինյանը պետք է հեռանա։",
    "Ադրբեջանը շարունակում է ագրեսիան։",
    "ՀԱՊԿ-ից դուրս գալը մեզ կմիացնի վտանգին։",
    "Փաշինյանը վնասում է պետական կառույցներին։",
    "Ադրբեջանը անտեսում է միջազգային իրավունքը։",
    "ՀԱՊԿ-ի օգնությունը անհրաժեշտ է։",
    "Փաշինյանի կառավարությունը ձախողվել է։",
    "Ադրբեջանական ագրեսիան աճում է։",
    "ՀԱՊԿ-ից դուրս գալը աղետալի կլինի։",
    "Փաշինյանը չի պաշտպանում մեր սահմանները։",
    "Ադրբեջանը շարունակում է սպառնալ։",
    "ՀԱՊԿ-ը պաշտպանում է մեզ։",
    "Փաշինյանը վտանգում է պետությունը։",
    "Ադրբեջանը ագրեսիվ քայլեր է կատարում։",
    "ՀԱՊԿ-ից դուրս գալը կբերի խուճապի։",
    "Փաշինյանը սխալ է վարում բանակցությունները։",
    "Ադրբեջանը չի պահպանում հրադադարը։",
    "ՀԱՊԿ-ից դուրս գալը հանցագործություն է։",
    "Փաշինյանը պարտություն է բերում։",
    "Ադրբեջանի սադրանքները խթանվում են։",
    "ՀԱՊԿ-ն ապահովում է պաշտպանությունը։",
    "Փաշինյանը չի մտածում ժողովրդի մասին։",
    "Ադրբեջանը չի հարգում մեր սահմանները։",
    "ՀԱՊԿ-ից դուրս գալը անիմաստ է։",
    "Փաշինյանը չի կարողանում բանակցել։",
    "Ադրբեջանի քայլերը ագրեսիվ են։",
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր հետևանքներ կունենա։",
    "Փաշինյանը խաթարում է խաղաղությունը։",
    "Ադրբեջանը շարունակում է սպառնալ խաղաղությանը։",
    "ՀԱՊԿ-ից դուրս գալը սխալ որոշում է։",
    "Փաշինյանը վտանգում է անվտանգությունը։",
    "Ադրբեջանը պլանավորում է ագրեսիա։",
    "ՀԱՊԿ-ն պաշտպանում է երկիրը։",
    "Փաշինյանի քայլերը վտանգավոր են։",
    "Ադրբեջանը խախտում է պայմանավորվածությունները։",
    "ՀԱՊԿ-ից դուրս գալը անխոհեմություն է։",
    "Փաշինյանը չի մտածում պետության ապագայի մասին։",
    "Ադրբեջանը սպառնում է Հայաստանի բնակչությանը։",
    "ՀԱՊԿ-ն միակ ելքն է։",
    "Փաշինյանը վտանգում է պետության հիմքերը։",
    "Ադրբեջանը խախտում է մեր իրավունքները։",
    "ՀԱՊԿ-ից դուրս գալը չի բերի խաղաղության։",
    "Փաշինյանը չի պաշտպանում սահմանները։",
    "Ադրբեջանը չի պահպանում խաղաղությունը։",
    "ՀԱՊԿ-ից դուրս գալը կբերի վտանգի։",
    "Փաշինյանը չի մտածում անվտանգության մասին։",
    "Ադրբեջանը ագրեսիվ պետություն է։",
    "ՀԱՊԿ-ից դուրս գալը աղետ կլինի։",
    "Փաշինյանը սխալ է մոտենում հարցերին։",
    "Ադրբեջանը խախտում է կարգը։",
    "ՀԱՊԿ-ից դուրս գալը վտանգավոր է։",
    "Փաշինյանը չի պաշտպանում երկիրը։",
    "Ադրբեջանը պլանավորում է հարձակում։"
]

neutral_texts = [
    "Երևանում այս շաբաթ արևոտ եղանակ է սպասվում։",
    "Հայաստանում տեղի կունենա երաժշտական փառատոն։",
    "Գյումրիում բացվել է նոր ցուցահանդես։",
    "Մարզիկները պատրաստվում են միջազգային մրցաշարին։",
    "Նոր ֆիլմի պրեմիերան մեծ հետաքրքրություն է առաջացրել։",
    "Դպրոցներում սկսվել են ամառային արձակուրդները։",
    "Երևանում բացվել է ժամանակակից արվեստի կենտրոն։",
    "Կենտրոնական պուրակում տնկվել են նոր ծառեր։",
    "Տեղի ունեցավ գարնանային տոնական միջոցառում։",
    "Հանրապետությունում արձանագրվել է զբոսաշրջության աճ։",
    "Մարզահրապարակում անցկացվեց մարզական փառատոն։",
    "Հանրային գրադարանում նոր գրքեր են ավելացվել։",
    "Կենտրոնական շուկայում նոր ապրանքատեսակներ են։",
    "Երևանի կենդանաբանական այգում ծնվել է նոր առյուծ։",
    "Հանրային այգում կազմակերպվեց մաքրման ակցիա։",
    "Շաբաթ օրը անցկացվեց հեծանվավազքի մրցույթ։",
    "Արցախում տեղի կունենա մշակութային համերգ։",
    "Տեղական թատրոնը ներկայացրեց նոր ներկայացում։",
    "Երևանի մետրոյում նոր վագոններ են գործարկվել։",
    "Սևանի ափին անցկացվեց զբոսաշրջային փառատոն։",
    "Հանրապետությունում տեղումներ չեն սպասվում։",
    "Ավանդական շուկայում մեծ թվով այցելուներ են եղել։",
    "Գեղարքունիքում անցկացվեց ձկնորսական մրցույթ։",
    "Հանրային այգում բացվեց նոր խաղահրապարակ։",
    "Մարզիկները մարզվում են ամեն օր։",
    "Մայրաքաղաքում բացվել է նոր սրճարան։",
    "Տեղի ունեցավ գարնանային տոնավաճառ։",
    "Երևանի փողոցները զարդարվել են։",
    "Գյուղական շրջաններում բերքահավաքը սկսվել է։",
    "Կենտրոնում անցկացվեց գրքի տոնավաճառ։",
    "Երաժշտական դպրոցում համերգ է եղել։",
    "Արագածոտնում բացվեց նոր զբոսաշրջային կենտրոն։",
    "Հանրապետությունում տաք եղանակ է սպասվում։",
    "Մարզադաշտում անցկացվեց ֆուտբոլային խաղ։",
    "Երևանի այգիներում նոր ծաղկամաններ են տեղադրվել։",
    "Ավելի քան 100 մարդ մասնակցեց միջոցառմանը։",
    "Գյուղերում սկսվել է խաղողի բերքահավաքը։",
    "Հանրային գրադարանում ընթերցանության օր է։",
    "Սևանում անցկացվեց ձմեռային փառատոն։",
    "Տեղի ունեցավ ֆիլմի փակ ցուցադրություն։",
    "Երևանի կենդանաբանական այգին այցելուներով լի էր։",
    "Գյումրու կենտրոնում նոր սրճարան է բացվել։",
    "Մայրաքաղաքում նոր շենքեր են կառուցվում։",
    "Տեղի ունեցավ քաղաքային մարաթոն։",
    "Երևանում հեծանվային մրցավազք էր։",
    "Սպորտային միջոցառումները շատացել են։",
    "Գյուղերում սկսվել է աշնանային բերքահավաքը։",
    "Երևանի բուսաբանական այգին այցելուներով լի էր։",
    "Տեղի ունեցավ արվեստի ցուցահանդես։",
    "Նոր թանգարան է բացվել մայրաքաղաքում։",
    "Կենտրոնական գրադարանում ընթերցանության օր էր։",
    "Տեղի ունեցավ պոեզիայի երեկո։",
    "Երևանում հեծանվորդների ակցիա էր։",
    "Սևանի ափին անցկացվեց ամառային տոնավաճառ։",
    "Երևանի այգիներում շատ զբոսաշրջիկներ կան։",
    "Կենտրոնում բացվեց գարնանային ցուցահանդես։",
    "Տեղի ունեցավ նոր ֆիլմի դիտում։",
    "Հանրապետությունում եղանակը տաքանում է։",
    "Գեղարքունիքում անցկացվեց երաժշտական փառատոն։",
    "Երևանի կենտրոնում բացվեց նոր գրախանութ։",
    "Արցախում անցկացվեց արվեստի ցուցահանդես։",
    "Տեղի ունեցավ գրքի շնորհանդես։",
    "Հանրային այգում երեխաների տոն էր։",
    "Երևանում անցկացվեց նորաձևության ցուցադրություն։",
    "Գյուղական տոնավաճառը շատ մարդ էր հավաքել։",
    "Կենտրոնում անցկացվեց մարզական միջոցառում։",
    "Երևանի թատրոնում նոր ներկայացում է։",
    "Տեղի ունեցավ ավանդական պարերի համերգ։",
    "Երևանում բացվեց նոր սրճարան։",
    "Հանրապետությունում անցկացվեց լուսանկարչական մրցույթ։",
    "Սևանում զբոսաշրջիկները շատացել են։",
    "Տեղի ունեցավ նկարչական ցուցահանդես։",
    "Երևանում բացվեց նոր կինոթատրոն։",
    "Հանրային այգում բացվեց ամառային սրճարան։",
    "Երևանում անցկացվեց տոնական շքերթ։",
    "Գեղարքունիքում անցկացվեց փառատոն։",
    "Հանրապետությունում եղանակը բարելավվել է։",
    "Տեղի ունեցավ երեխաների ցուցահանդես։",
    "Երևանում բացվեց տեխնոլոգիական կենտրոն։",
    "Տեղի ունեցավ գարնանային ցուցահանդես։",
    "Երևանում անցկացվեց հեծանվային արշավ։",
    "Գյուղերում սկսվել է աշնան բերքահավաքը։",
    "Հանրային այգում բացվեց նոր ծաղկանոց։",
    "Երևանում անցկացվեց գրականության օր։",
    "Տեղի ունեցավ ավանդական փառատոն։",
    "Երևանում բացվեց արվեստի սրահ։",
    "Հանրապետությունում մշակութային միջոցառումները շատացել են։",
    "Գյուղերում իրականացվում են տոնական միջոցառումներ։",
    "Տեղի ունեցավ ամառային բացօթյա համերգ։",
    "Երևանում բացվեց նոր գիտական կենտրոն։",
    "Տեղի ունեցավ երիտասարդական համաժողով։",
    "Հանրապետությունում մարզական կյանքը զարգանում է։",
    "Երևանի թատրոնում նոր բեմադրություն է։",
    "Տեղի ունեցավ նորաձևության ցուցահանդես։",
    "Երևանում բացվեց նոր այգի։",
    "Տեղի ունեցավ երիտասարդական փառատոն։",
    "Հանրապետությունում եղանակը տաքացել է։"
]

data = pd.DataFrame({
    "text": propaganda_texts + neutral_texts,
    "label": [1] * len(propaganda_texts) + [0] * len(neutral_texts)
})

vectorizer = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    token_pattern=r'\b[ա-ֆԱ-Ֆ]{2,}\b'
)

X = vectorizer.fit_transform(data["text"])
y = data["label"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

model = CatBoostClassifier(
    iterations=500,
    depth=5,
    learning_rate=0.03,
    verbose=0
)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("=== Գնահատման Արդյունքներ ===")
print(classification_report(y_test, y_pred, target_names=["Չեզոք", "Քարոզչական"]))
print("Ճշգրտություն:", accuracy_score(y_test, y_pred))

joblib.dump({"model": model, "vectorizer": vectorizer}, "armenian_propaganda_detector.pkl")
print("\nՄոդելը պահպանվել է armenian_propaganda_detector.pkl ֆայլում։")


In [ ]:
import joblib

model_data = joblib.load("armenian_propaganda_detector.pkl")
model = model_data["model"]
vectorizer = model_data["vectorizer"]

def predict_sentence(sentence: str):
    """Վերադարձնում է 'Քարոզչական' կամ 'Չեզոք' տեքստի դասը։"""
    X = vectorizer.transform([sentence])
    pred = model.predict(X)[0]
    return "Քարոզչական" if pred == 1 else "Չեզոք"

if __name__ == "__main__":
    sentence = input("Գրիր նախադասությունը ստուգելու համար: ")
    result = predict_sentence(sentence)
    print(f"Արդյունք: {result}")


In [ ]:
!pip install transformers datasets sentencepiece evaluate nltk
import nltk
nltk.download("punkt")
nltk.download("wordnet")

import random
import torch
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
import evaluate
import pandas as pd
import re

# Use the existing data DataFrame from the notebook
# Assuming 'data' DataFrame with 'text' and 'label' columns exists from previous cells.
# If not, you might need to load or define it here.
# For example:
# data = pd.read_csv("your_dataset.csv")

# Add the clean_text function
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    return text

# Apply cleaning
data['cleaned_text'] = data['text'].apply(clean_text)

# Split data and save to csv
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42, stratify=data['label'])
train_df.to_csv("train.csv", index=False)
test_df.to_csv("test.csv", index=False)

# Load the dataset from the saved csv files
dataset = load_dataset("csv", data_files={"train": "train.csv", "test": "test.csv"})


from nltk.corpus import wordnet

def synonym_replacement(text, n=1):
    words = text.split()
    new_words = words.copy()
    random_word_list = list(set(words))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = wordnet.synsets(random_word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:
            break
    return " ".join(new_words)

def augment_examples(example):
    # Apply augmentation to the cleaned text
    if random.random() < 0.3:
        example["cleaned_text"] = synonym_replacement(example["cleaned_text"])
    return example

# Apply augmentation on the cleaned text column
augmented_train = dataset["train"].map(augment_examples)

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize(batch):
    # Tokenize the cleaned text
    return tokenizer(batch["cleaned_text"], truncation=True)

tokenized_train = augmented_train.map(tokenize, batched=True)
tokenized_test = dataset["test"].map(tokenize, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(-1)
    return accuracy.compute(predictions=preds, references=labels)

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.save_model("./AzgIntel-XLMR")
tokenizer.save_pretrained("./AzgIntel-XLMR")

In [ ]:
# Պահանջվող գրադարանները
!pip install datasets

from datasets import load_dataset
import pandas as pd

# Ներբեռնում ենք azcorpus_v0 dataset-ը
dataset = load_dataset("azcorpus/azcorpus_v0")

# Տեսնենք ինչ ենթաբաժիններ ունի
print(dataset)

# Օրինակ վերցնենք լուրերի բաժինը (news)
news_data = dataset["news"]

# Դիտենք առաջին մի քանի լուրերը
print(news_data[0])

# Վերափոխենք pandas DataFrame-ի, որպեսզի հեշտ ֆիլտրենք
df = pd.DataFrame(news_data)

# Ֆիլտրենք միայն քաղաքական լուրերը
# այստեղ "siyasət" (քաղաքականություն) հիմնական keyword-ն է ադրբեջաներենում
politics_df = df[df["text"].str.contains("siyasət|politika|parlament|prezident|hakimiyyət", case=False, na=False)]

# Տեսնենք քանի հոդված ստացանք
print("Քաղաքական լուրերի քանակ:", len(politics_df))

# Պահպանենք CSV ձևաչափով՝ հետագա օգտագործման համար
politics_df.to_csv("azerbaijani_political_news.csv", index=False, encoding="utf-8-sig")


In [ ]:
!pip install huggingface_hub datasets

from huggingface_hub import login

# Կպցրու քո HF access token-ը այստեղ
login("hf_xxxxxxxxxxxxxxxxxxxxxxxxx")
